# SEC EDGAR Pipeline API:
An ETL Pipleine built with Bonobo that ingests all of the SEC EDGAR Filings data for a specific stock.

### Step 1: Create logic that constructs a url for the EDGAR filings based on input parameters:
```python
build_edgar_url("AAPL", "10-K", max=100) -> "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000320193&type=10-K&dateb=&owner=exclude&count=100&search_text=
```

In [1]:
# Importing necessary libraries:
import pandas as pd
import sqlite3
import requests
from bs4 import BeautifulSoup
from ETL_pipelines.base_pipeline import Pipeline

# Local SQLITE database for testing:
test_dbpath = "../ETL_pipelines/stock_pipeline/test.sqlite"

# Local ticker text file for testing:
ticker_txt = "../ETL_pipelines/stock_pipeline/example.txt"

## Building the main pipeline object:

In [2]:
class EDGARFilingsPipeline(Pipeline):
    """
    ADD DOCUMENTATION
    """
    def __init__(self, dbpath):
        
        # Declaring instance params:
        self.ticker_lst = []
        self.base_sec_url = "https://www.sec.gov"
        self.dbpath = dbpath
        
    def extract_filings_page(self):
        """Method performs the 
        """
        con = sqlite3.connect(self.dbpath)
        
        # Performing a query to the local database for the CIK:
        SPY_df = pd.read_sql_query("SELECT * FROM SPY_components", con)
                
        # Iterating through the ticker list to perform EDGAR query for each:
        for ticker in self.ticker_lst:
            
            # Searching existing database listings for CIK data:
            ticker_row = SPY_df.loc[SPY_df["Symbol"] == ticker]
            ticker_cik_arr = ticker_row['CIK'].array
            
            # If ticker CIK cannot be found internally, performing external request:
            if len(ticker_cik_arr) == 0:
                pass # TODO: Add Error Catch.
            
            # Ticker CIK has been found. Continue logic:
            else:
                ticker_cik = ticker_cik_arr[0]
                
                # Performing Request to SEC EDGAR:
                edgar_result_url = self._build_edgar_url(cik=ticker_cik, filings_type="10-K")
                edgar_result_response = requests.get(edgar_result_url)
                
                # Adding conditonal statements to catch response error:
                if edgar_result_response.status_code != 200:
                    pass # TODO: Add Error Catch
                
                else:
                    # Converting the response content to BeautifulSoup and parsing:
                    edgar_soup = BeautifulSoup(edgar_result_response.text, "html.parser")
                    filings_table = edgar_soup.find("table", {"class":"tableFile2"})
                    
                         # Building list of urls for each filing:
                    filings_urls = [
                        f"{self.base_sec_url}{href['href']}" for href in filings_table.find_all(
                        "a", {"id":"documentsbutton"}, href=True)]
                    
                    # Constructing a dataframe out of html content:
                    filings_df_lst = pd.read_html(str(filings_table))
                    filings_df = pd.DataFrame(filings_df_lst[0])
                    
                    # Adding the urls to each individual filings to df: 
                    filings_df["Format"] = filings_urls
                    filings_df.set_index("Filing Date", inplace=True)
                    
                    # Performing search of internal database for existing filings:
                    unique_filings_df = self._build_unique_filings(filings_df, ticker)
                    
                    yield (ticker, unique_filings_df)
    
    
    def build_ticker_lst(self, filepath):
        """
        """
        self._ticker_filepath = filepath
        
        # Opening and exracting the information from the 
        ticker_file = open(self._ticker_filepath, 'rt')
        file_contents = ticker_file.read()
        
        # Seperating string into single list elements: 
        split_ticker_str = file_contents.split("\n")
        
        # Assigning split ticker list to the main param:
        self.ticker_lst = split_ticker_str
        
        ticker_file.close()
        
        # Converting the ticker list to a set and back to extract only unique elements:
        self.ticker_lst = list(set(self.ticker_lst))
        
        
    def _build_edgar_url(self, cik="", filings_type="", prior_to="", ownership="", no_of_entries=100):
        """
        """
        # Constructing the EDGAR query based on the search params:
        edgar_search_url = f"{self.base_sec_url}/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type={filings_type}&dateb={prior_to}&owner={ownership}&count={str(no_of_entries)}"
        return edgar_search_url
    
    def _build_unique_filings(self, df, ticker):
        """TODO: ADD Documentation.
        
        - Make SQL Request for table containing filing data for specific ticker 
        - Parsing SQL table looking for entries that are also present in the input dataframe. 
        - Removing entries in the input df that are already present in the database.
        """
        # Querying database for data table:
        tbl_name = f"{ticker}_filings"
        tbl_query = f"SELECT * FROM {tbl_name}"
        
        # Try-Catch to declare db_tbl as None if it does not exist:
        try:
            database_tbl = pd.read_sql_query(tbl_query, con)
        except:
            database_tbl = None
            
        # Conditional determining if dataframes need to be compared:
        if database_tbl == None:
            return df
        
        # TODO: Write database comparing methods:
        else:
            # Comparing the two dataframes for unique elements:
            for index, row in df.iterrows():
                print(row)
                
    def _extract_individual_filing(url):
        """Method ingests a url to the SEC EDGAR webpage that
        lists an individual filing and returns the html content
        of said filing.

        This html content is extracted from the href of the first
        element in the results table.

        TODO: Fully articulate this documentation.
        """
        # Performing request to the Filing Detail Page:
        response = requests.get(url)

        # Converting the html content to the soup object and parsing:
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            filing_detail_tbl = soup.find("table", {"class":"tableFile"})

            # Try-Catch block meant to catch break in Filing Detail page structure:
            try:
                # Searching for first href in the Filing Detail Table:
                document_href = filing_detail_tbl.find_all("a", href=True)[0]["href"]
                document_href = document_href.replace("/ix?doc=", "") # Dropping XBRL label for only HTML document

                # Adding the SEC url to the href to build url to actual filing content:
                document_url = f"https://www.sec.gov{document_href}"

                # Performing the Request to the document_url for filing content:
                filing_content = requests.get(document_url).text

                return filing_content

            except:
                return None

        else:
            raise ValueError(f"Response Status Code for {url} is {response.status_code}")


    
# Declaring Example Test Pipeline:
test_pipeline = EDGARFilingsPipeline(test_dbpath)

### EDGARFilingsPipeline Object Builds the Correct EDGAR Search URL:

In [3]:
print(test_pipeline._build_edgar_url(cik= '0000320193', filings_type="10-K"))

https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000320193&type=10-K&dateb=&owner=&count=100


### EDGARFilingsPipeline Object Constructs Ticker List from Text File:

In [4]:
print("Ticker list before build method:", test_pipeline.ticker_lst)
test_pipeline.build_ticker_lst(ticker_txt)
print("Ticker list after build method:", test_pipeline.ticker_lst)

Ticker list before build method: []
Ticker list after build method: ['SPY', 'XOM', 'TSLA', 'ICLN']


### Method that extracts the main document from a Filing Detail page link:

In [5]:
def _extract_individual_filing(url):
    """Method ingests a url to the SEC EDGAR webpage that
    lists an individual filing and returns the html content
    of said filing.
        
    This html content is extracted from the href of the first
    element in the results table.
        
    TODO: Fully articulate this documentation.
    """
    # Performing request to the Filing Detail Page:
    response = requests.get(url)
    
    # Converting the html content to the soup object and parsing:
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        filing_detail_tbl = soup.find("table", {"class":"tableFile"})
        
        # Try-Catch block meant to catch break in Filing Detail page structure:
        try:
            # Searching for first href in the Filing Detail Table:
            document_href = filing_detail_tbl.find_all("a", href=True)[0]["href"]
            document_href = document_href.replace("/ix?doc=", "") # Dropping XBRL label for only HTML document
            
            # Adding the SEC url to the href to build url to actual filing content:
            document_url = f"https://www.sec.gov{document_href}"
           
            # Performing the Request to the document_url for filing content:
            filing_content = requests.get(document_url).text
            
            return filing_content
        
        except:
            return None
            
    else:
        raise ValueError(f"Response Status Code for {url} is {response.status_code}")

# Example implementation of method: 
#print(_extract_individual_filing("https://www.sec.gov/Archives/edgar/data/34088/000003408820000016/0000034088-20-000016-index.htm"))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### EDGARFilingsPipeline Objects Performs Extraction of EDGAR Search Papers for each Ticker:

In [6]:
test_pipeline.extract_filings_page()

<generator object EDGARFilingsPipeline.extract_filings_page at 0x7ffab515d890>

### Method that compares two dataframes that have the same index and extracts the unique elements:

In [18]:
def build_unique_dataframe(df1, df2):
    """Method compares two dataframes with the same index and
    drops the rows that are duplicates. It then returns a dataframe
    containing only the unique elements.
    """
    index_a = df1.index
    index_b = df2.index
    
    unique_index = list((set(index_a) | set(index_b)) - (set(index_a) & set(index_b)))    
    
    df = df2.loc[unique_index]
    print(df)
    
# Intialise data to Dicts of series. 
dict1 = {'one' : pd.Series([10, 20, 30, 40], 
                       index =['a', 'b', 'c', 'd']), 
      'two' : pd.Series([10, 20, 30, 40], 
                        index =['a', 'b', 'c', 'd'])} 

# Intialise data to Dicts of series. 
dict2 = {'one' : pd.Series([10, 20, 30, 40], 
                       index =['a', 'b', 'c', 'd']), 
      'three' : pd.Series([10, 20, 30, 40], 
                        index =['a', 'b', 'c', 'd'])}

df1 = pd.DataFrame(dict1)
df2 = pd.DataFrame(dict2)

build_unique_dataframe(df1, df2)

Empty DataFrame
Columns: [one, three]
Index: []
